In [436]:
from calendar import month_name
import numpy as np
import xarray as xr
from pathlib import Path
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.core.properties import value
from bokeh.transform import dodge
from bokeh.models import Label
from bokeh.palettes import Accent6, Dark2, Category10
from bokeh.colors.named import darkgray

In [437]:
output_notebook()

Loading BokehJS ...

In [438]:
# Define dataset file paths
DATA_DIR = Path('.')
DATASETS = {
    'l8': DATA_DIR / 'l8_wofs.nc',
    's1': DATA_DIR / 'wofs_s1.nc',
    's2': DATA_DIR / 'wofs_s2ab.nc'
}

In [439]:
# Load datasets
datasets = {name: xr.open_dataset(path) for name, path in DATASETS.items()}
print('Loaded datasets: {}'.format(', '.join(datasets.keys())))

Loaded datasets: l8, s1, s2


In [440]:
# For testing: drop January values in XX
# datasets['XX'] = datasets['XX'].sel(time = slice('2017-02-01', '2018-01-01')).time

In [441]:
# Calculate frequencies per months. Months with no data are handled by combining with
# an empty `all_months` data array. 
coords={'month': range(1, 13)}
months_str = [m[:3] for m in list(month_name)[1:]]
frequencies = {}
for name, dataset in datasets.items():
    freq = dataset.time.groupby('time.month').count('time')
    all_months = xr.DataArray(data = [0]*12, coords=coords, dims=coords, name='time')
    frequencies[name] = freq.combine_first(all_months).assign_coords(month=months_str)

In [442]:
# Calculate the aggregated value for each month
aggregated = np.sum([freq.values for freq in frequencies.values()], axis=0)
aggregated

array([ 5,  8,  7,  8,  7,  6, 11,  9, 11,  9, 10, 11])

In [443]:
# Add a scatter plot on top of the bar chart, but since they don't share the same X axis, we use
# labels (bullet character) and map them the best we can on top of the existing figure. This requires
# to manually calibrate the X and Y bounds based on the figure size.
X_BOUNDS = (33, 690)
Y_BOUNDS = (-23, 300)
def add_point(p, doy, y, colour='red', alpha=1.0):
    x = X_BOUNDS[0] + X_BOUNDS[1] * (doy - 1) / 365
    y = Y_BOUNDS[0] + Y_BOUNDS[1] * y
    citation = Label(x=x, y=y, x_units='screen', y_units='screen',
                     text='•', render_mode='css',
                     text_color=colour, text_font_size='30pt', text_alpha=alpha,
                     border_line_color='black', border_line_alpha=0.0,
                     background_fill_color='white', background_fill_alpha=0.0)
    p.add_layout(citation)

In [444]:
# Generate the bar chart on a new figure
data = {name: freq.values for name, freq in frequencies.items()}
data['months'] = months_str
data['aggregated'] = aggregated

# Calculate the max frequency to scale the Y-axis automatically
max_freq = max([freq.values.max() for freq in frequencies.values()] + [aggregated.max()])

p = figure(x_range=months_str,
           y_range=(0, max_freq),
           plot_width=800,
           plot_height=400,
           title="Monthly data points",
           toolbar_location=None,
           tools="")

# Calculate offsets and widths for the bars
inter_group = 0.4
width = (1 - inter_group / 2) / len(frequencies)
offset = (width + inter_group / 2 - 1) / 2

# Plot the aggregate bar
p.vbar(x=dodge('months', 0, range=p.x_range), 
       top='aggregated', 
       width=(1 - inter_group + width / 2), 
       source=data,
       color='#000000', 
       legend=value('aggregated'), 
       fill_alpha=0, 
       line_color='lightgray', 
       line_width=2.0)

# Plot the raw data bars, one per dataset
for count, name in enumerate(frequencies.keys()):
    p.vbar(x=dodge('months', count*width+offset, range=p.x_range), 
           top=name, 
           width=width-0.1, 
           source=data,
           color=Category10[10][count],
           legend=value(name), 
           fill_alpha=.4)
    
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

In [445]:
# Plot WOFS values on same figure
# Points are calculated as number of days from that day, and to a max of 365 days
REFERENCE_DATE = np.datetime64('2016-12-31')
count = 0
for name, dataset in datasets.items():
    # Calculate day of year
    doys = (np.array(dataset.time.values, dtype='M8[D]') - REFERENCE_DATE) / np.timedelta64(1,'D')
    wofs = dataset.mean(dim=['x','y']).wofs.values * 2.2  # Add some scaling for now, until they share the same scale
    for doy, wof in zip(doys, wofs):
        if not np.isnan(wof):  # Only print of wofs is not NaN
            add_point(p, doy, wof, Category10[10][count])
    count += 1
    
# # Calibration points
# add_point(p, 1, 0)
# add_point(p, 365, 1.0)

show(p)